In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [3]:

from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")

In [4]:
question="what is AI "

In [5]:
print(llm.invoke(question).content)

**Artificial Intelligence (AI)**

Artificial Intelligence refers to the simulation of human intelligence processes by machines, particularly computer systems. It encompasses a wide range of techniques and approaches aimed at enabling machines to perform tasks that typically require human intelligence, such as:

* **Learning and Adaptation:** AI systems can learn from data, identify patterns, and adapt their behavior based on new information.
* **Reasoning and Problem-Solving:** They can analyze complex information, draw logical inferences, and make decisions.
* **Natural Language Processing (NLP):** AI systems can understand, interpret, and generate human language.
* **Computer Vision:** They can analyze and interpret visual data (e.g., images, videos).
* **Speech Recognition and Synthesis:** They can recognize spoken words and generate synthesized speech.

**Types of AI:**

* **Machine Learning (ML):** AI systems that learn from data without explicit programming.
* **Deep Learning:** 

### **Simple AI Assistant**

In [6]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [7]:
store={}

In [8]:

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [9]:
config = {"configurable": {"session_id": "firstchat"}}

In [10]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [11]:
model_with_memory.invoke(("Hi! I'm Sureshbeekhani"),config=config).content

'Hello, Sureshbeekhani. How can I assist you today?'

In [12]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content

'You did not provide me with your name, so I cannot tell you what it is.'

In [13]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Sureshbeekhani", additional_kwargs={}, response_metadata={}), AIMessage(content='Hello, Sureshbeekhani. How can I assist you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-2b4414ea-55fe-4279-bf76-b01ae8948087-0', usage_metadata={'input_tokens': 9, 'output_tokens': 13, 'total_tokens': 22, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='tell me what is my name?', additional_kwargs={}, response_metadata={}), AI

### **RAG with LCEL**

In [14]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [15]:
### Reading the txt files from source directory

loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [16]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)


In [17]:

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [18]:
question ="what is llama?"
print(retrieval_chain.invoke(question))

A family of large language models developed by Meta AI


### **Let's Start with Tools and Agents**

In [19]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [20]:
api_wrapper=WikipediaAPIWrapper()

In [21]:
tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [22]:
tool.name

'wikipedia'

In [23]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [24]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [25]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Milvus (vector database)
Summary: Milvus is a distributed vector database developed by Zilliz. It is available as both open-source software and a cloud service.
Milvus is an open-source project under LF AI & Data Foundation distributed under the Apache License 2.0.

Page: Retrieval-augmented generation
Summary: Retrieval-Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment informa

In [26]:
from langchain_community.tools import YouTubeSearchTool

In [27]:
tool2=YouTubeSearchTool()

In [28]:
tool2.name

'youtube_search'

In [29]:
tool2.run("Irfan Malik")

"['https://www.youtube.com/watch?v=YIOfTrhEdEg&pp=ygULSXJmYW4gTWFsaWs%3D', 'https://www.youtube.com/watch?v=BDEiT-IJTGI&pp=ygULSXJmYW4gTWFsaWs%3D']"

In [30]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [31]:
tool3=TavilySearchResults()

In [32]:
tool.invoke({"query": "What happened in the latest burning man floods"})

'Page: Burning Man\nSummary: Burning Man is a week-long large-scale desert event focused on "community, art, self-expression, and self-reliance" held annually in the western United States. The event\'s name comes from its culminating ceremony: the symbolic burning of a large wooden effigy, referred to as the Man, that occurs on the penultimate night, the Saturday evening before Labor Day. Since 1990, the event has been at Black Rock City in northwestern Nevada, a temporary city erected in the Black Rock Desert about 100 miles (160 km) north-northeast of Reno. According to Burning Man co-founder Larry Harvey in 2004, the event is guided by ten stated principles: radical inclusion, gifting, decommodification, radical self-reliance, radical self-expression, communal effort, civic responsibility, leaving no trace, participation, and immediacy.\nBurning Man features no headliners or scheduled performers; participants create all the art, activities, and events. Artwork includes experimental 

In [33]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent

In [34]:
tool=load_tools(["tool3"],llm=llm)

In [35]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

C:\Users\SURESH BEEKHANI\AppData\Local\Temp\ipykernel_12740\2645007812.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [36]:
agent.run("What is current GDP of pakistan?")

C:\Users\SURESH BEEKHANI\AppData\Local\Temp\ipykernel_12740\3358457773.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("What is current GDP of pakistan?")




> Entering new AgentExecutor chain...
Action: [wikipedia]
Action Input: GDP of Pakistan
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: GDP of Pakistan
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: Economy of Pakistan
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: GDP of Pakistan
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: Economy of Pakistan
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: Economy of Pakistan
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Final Answer: [wikipedia] is not a valid tool, try one of [wikipedia].

> Finished chain.


'[wikipedia] is not a valid tool, try one of [wikipedia].'